In [1]:
import pandas as pd
from os import mkdir, listdir
from os.path import join as pJoin
from shutil import copyfile
import random as rnd
from math import floor

In [2]:
def decompose_line(line):
    decompose = line.split('|')
    tokens = []
    for str in decompose:
        if len(str) > 1:
            tokens.append(str.strip())
    return tokens

In [3]:
data_dir = '/media/anubis/Data/OpenVINO/PH2Dataset'
class_dir = '/media/anubis/Data/OpenVINO/Classification'
im_size = (50, 75, 3)

diagnosis_file = open(pJoin(data_dir, 'PH2_dataset.txt'), 'r')
header = decompose_line(diagnosis_file.readline())
t = []
for line in diagnosis_file:
    if "||" in line:
        t.append(decompose_line(line))        
diagnosis_file.close()
df_dataset = pd.DataFrame(t, columns = header)

In [4]:
class_number = len(df_dataset['Clinical Diagnosis'].unique())
print(class_number, 'classes available')
df_dataset

3 classes available


,Name,Histological Diagnosis,Clinical Diagnosis,Asymmetry,Pigment Network,Dots/Globules,Streaks,Regression Areas,Blue-Whitish Veil,Colors
0,IMD003,,0,0,T,A,A,A,A,4
1,IMD009,,0,0,T,A,A,A,A,3
2,IMD016,,0,0,T,T,A,A,A,3 4
3,IMD022,,0,0,T,A,A,A,A,3
4,IMD024,,0,0,T,A,A,A,A,3 4
...,...,...,...,...,...,...,...,...,...,...
195,IMD424,Melanoma,2,2,AT,AT,P,A,P,3 4 5 6
196,IMD425,Melanoma,2,2,AT,AT,A,P,P,3 4 5 6
197,IMD426,Melanoma,2,2,AT,A,A,P,A,3 4
198,IMD429,,2,0,AT,A,P,A,P,3 4 5 6


In [5]:
try: 
    mkdir(class_dir)
    print("Directory" , class_dir ,  "Created")
except OSError as error: 
    print('Directory already exists')

for c in range(class_number):
    try:
        mkdir(pJoin(class_dir, str(c)))
    except OSError as error:
        print('Directory', str(c), 'already exists')

Directory /media/anubis/Data/OpenVINO/Classification Created


In [6]:
dataset_dir = "/media/anubis/Data/OpenVINO/PH2Dataset/PH2 Dataset images"
ImageNames = listdir(dataset_dir)
FormatFile = ".bmp"

for i in range(len(df_dataset)):
    Image = df_dataset.loc[i][0]
    directory = df_dataset.loc[i][2]
    ImageSrcFile = pJoin(dataset_dir, Image, Image + "_Dermoscopic_Image", Image + FormatFile)
    ImageDstFile = pJoin(class_dir, directory, Image + FormatFile)
    copyfile(ImageSrcFile, ImageDstFile)

In [7]:
class_dir = '/media/anubis/Data/OpenVINO/Classification'
val_dir = '/media/anubis/Data/OpenVINO/Classification/validation'
train_dir = '/media/anubis/Data/OpenVINO/Classification/train'
try:
    mkdir(val_dir)
    mkdir(train_dir)
    for c in range(class_number):
        try:
            mkdir(pJoin(val_dir, str(c)))
            mkdir(pJoin(train_dir, str(c)))
        except OSError as error:
            print('DAE')
except OSError as error:
            print('DAE')
    

for i in range(class_number):
    image_dir = pJoin(class_dir, str(i))
    image_names = listdir(image_dir)
    rnd.shuffle(image_names)
    image_qnt = len(image_names)
    cut_qnt = floor(image_qnt*.7)
    toTrain = image_names[:cut_qnt]
    toVal = image_names[cut_qnt:]
    for Image in toVal:
        ImageSrc = pJoin(class_dir, str(i), Image)
        ImageDst = pJoin(val_dir, str(i), Image)
        copyfile(ImageSrc, ImageDst)
        
    for Image in toTrain:
        ImageSrc = pJoin(class_dir, str(i), Image)
        ImageDst = pJoin(train_dir, str(i), Image)
        copyfile(ImageSrc, ImageDst)